In [1]:
import ast

In [19]:
path = "/Users/yangxinyi/Downloads/200_临时文件夹/english_test_project/config.py"
with open(path) as f:
    source_code = f.read()

In [20]:
tree = ast.parse(source_code, mode='exec')

In [21]:
print(ast.dump(tree, indent=4))

Module(
    body=[
        Import(
            names=[
                alias(name='json')]),
        Import(
            names=[
                alias(name='logging')]),
        Import(
            names=[
                alias(name='os')]),
        Import(
            names=[
                alias(name='pickle')]),
        ImportFrom(
            module='common.log',
            names=[
                alias(name='logger')],
            level=0),
        Import(
            names=[
                alias(name='pandas', asname='pd')]),
        ImportFrom(
            module='translate',
            names=[
                alias(name='factory')],
            level=0),
        ImportFrom(
            module='translate.baidu.baidu_translate',
            names=[
                alias(name='BaiduTranslator')],
            level=0),
        Expr(
            value=Call(
                func=Name(id='print', ctx=Load()),
                args=[
                    Name(id='factory', ctx=Load()

2. 遍历 AST

ast.NodeVisitor 和 ast.NodeTransformer 类可以被用来遍历和修改 AST。NodeVisitor 用于遍历树，而 NodeTransformer 允许你修改树。

例如，以下是如何使用 NodeVisitor 来打印所有的二元运算符：

In [22]:
class Visitor(ast.NodeVisitor):
    def visit_BinOp(self, node):
        print(f"Found a binary operation: {ast.dump(node)}")
        self.generic_visit(node)  # 继续遍历子节点

In [24]:
visitor = Visitor()
visitor.visit(tree)

In [12]:
class IncrementNumbers(ast.NodeTransformer):
    def visit_Constant(self, node):  # 在 Python 3.8+ 中，数字是使用 Constant 节点而非 Num
        if isinstance(node.value, int):  # 确保只修改整数
            return ast.Constant(value=node.value + 1)
        return node

# 解析源代码到 AST
source_code = "x = 1 + 2"
tree = ast.parse(source_code, mode='exec')

# 修改 AST
transformer = IncrementNumbers()
modified_tree = transformer.visit(tree)
ast.fix_missing_locations(modified_tree)  # 自动修复缺少的行号和列偏移

# 编译并执行修改后的 AST
exec(compile(modified_tree, filename="<ast>", mode="exec"))

In [14]:
# 假设 modified_tree 已经是你通过 NodeTransformer 修改过的 AST
source_code_modified = ast.unparse(modified_tree)
print(source_code_modified)

x = 2 + 3


In [13]:
import astor

# 假设 modified_tree 已经是你通过 NodeTransformer 修改过的 AST
source_code_modified = astor.to_source(modified_tree)
print(source_code_modified)

x = 2 + 3



In [25]:
from redbaron import RedBaron

# 原始代码
code = '''
def add_numbers(a, b):
    # 注释内容
    result = a + b
    return result
'''

# 翻译字典
translation_dict = {
    'add_numbers': '添加数字',
    'a': '数字一',
    'b': '数字二',
    'result': '结果'
}

In [26]:
# 解析代码
red = RedBaron(code)

In [33]:
from redbaron import RedBaron
import re

code = '''
# 这是一个计算器类
class Calculator:
    def add_numbers(self, a, b):
        # 计算两个数字的和
        result = a + b
        return result
'''

translation_dict = {
    'Calculator': '计算器',
    'add_numbers': '添加数字',
    'self': '自身',
    'a': '数字一',
    'b': '数字二',
    'result': '结果',
    'numbers': '数字',
    'calculate': '计算',
    'sum': '和'
}

red = RedBaron(code)

def translate_comment(comment_text):
    # 使用正则表达式替换注释中的标识符
    for eng, cn in translation_dict.items():
        comment_text = re.sub(r'\b' + re.escape(eng) + r'\b', cn, comment_text)
    return comment_text

def replace_names(node):
    if node.type == 'name':
        if node.value in translation_dict:
            node.value = translation_dict[node.value]
    elif node.type == 'def':
        if node.name in translation_dict:
            node.name = translation_dict[node.name]
        # 处理参数列表
        for arg_node in node.arguments:
            if arg_node.type == 'name':
                if arg_node.value in translation_dict:
                    arg_node.value = translation_dict[arg_node.value]
            elif arg_node.type == 'def_argument':
                if arg_node.target.value in translation_dict:
                    arg_node.target.value = translation_dict[arg_node.target.value]
                if arg_node.value is not None:
                    replace_names(arg_node.value)
            elif arg_node.type == 'def_parameter':
                if arg_node.name.value in translation_dict:
                    arg_node.name.value = translation_dict[arg_node.name.value]
                if arg_node.annotation is not None:
                    replace_names(arg_node.annotation)
        # 递归处理函数体
        for child in node.value:
            replace_names(child)
    elif node.type == 'class':
        if node.name in translation_dict:
            node.name = translation_dict[node.name]
        # 递归处理类体
        for child in node.value:
            replace_names(child)
    elif node.type == 'assign':
        for target in node.targets:
            replace_names(target)
        if hasattr(node, 'value'):
            replace_names(node.value)
    elif node.type == 'call':
        # 处理函数调用的参数
        for arg_node in node.arguments:
            replace_names(arg_node)
    elif node.type == 'attribute':
        # 处理属性访问
        if hasattr(node, 'value'):
            replace_names(node.value)
        if node.attr in translation_dict:
            node.attr = translation_dict[node.attr]
    elif node.type == 'comment':
        node.value = translate_comment(node.value)
    # 递归处理其他子节点
    if hasattr(node, 'value') and isinstance(node.value, list):
        for child in node.value:
            replace_names(child)
    if hasattr(node, 'else_') and node.else_ is not None:
        replace_names(node.else_)
    if hasattr(node, 'children'):
        for child in node.children:
            replace_names(child)

# 遍历语法树，替换变量名和注释内容
for node in red:
    replace_names(node)

# 生成新的代码
new_code = red.dumps()
print(new_code)


# 这是一个计算器类
class 计算器:
    def 添加数字(自身, 数字一, 数字二):
        # 计算两个数字的和
        result = a + b
        return result

